In [23]:
import utils
from jsonl_dump import download_parse_delivered_into_jsonl
from utils import DATA_DIR
LIMIT_ITEMS_TO_FIRST_N = 50 # None for full
BATCH_IDS = [1,2,3,4]
max_workers = 20

In [24]:
delivered4 = download_parse_delivered_into_jsonl(BATCH_IDS, no_work=False)

Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Filling out missing header...
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 100%.
Download progress: 1

In [25]:
delivered_df = delivered4['delivered_df']
conversations = delivered4['conversations']

In [26]:
from importlib import reload
import use_case
reload(use_case)
from use_case import get_use_case_data_batch_conversations
import areas_of_focus
reload(areas_of_focus)
from areas_of_focus import get_areas_of_focus_data_batch_conversations
import behaviour_tagging
reload(behaviour_tagging)
from behaviour_tagging import get_behavioural_tags_data_batch_conversations

In [27]:
from utils import DATA_DIR
import os
import pickle
from datetime import datetime

batch_folder_prefix = DATA_DIR + 'jsonl_conversations/batch_'
progress_backup_folder = DATA_DIR + 'progress_backup/'
current_date = datetime.now().strftime('%Y-%m-%d__%H-%M-%S')

batch_results = {}
item_types = ['behavioural_tags', 'use_case_data', 'areas_of_focus_data']

number_of_batches_total = len(BATCH_IDS)
for i in range(1, 1 + number_of_batches_total):
    print('='*60)
    print(f"Processing batch {i} of {number_of_batches_total}")
    batch_folder = batch_folder_prefix + str(i)
    batch_results[i] = {}
    
    print("  Generating behavioural tags...")
    batch_results[i]['behavioural_tags'] = get_behavioural_tags_data_batch_conversations(
        batch_folder, max_workers=max_workers, limit_items_to_first_n=LIMIT_ITEMS_TO_FIRST_N
    )
    # Save progress after each item
    progress_backup_path = os.path.join(progress_backup_folder, f'{current_date}.pkl')
    os.makedirs(os.path.dirname(progress_backup_path), exist_ok=True)
    with open(progress_backup_path, 'wb') as backup_file:
        pickle.dump(batch_results, backup_file)
    
    print("  Generating use case data...")
    # Assuming similar functions exist for use_case_data and areas_of_focus_data
    # and they follow the same function signature as get_behavioural_tags_data_batch_conversations
    batch_results[i]['use_case_data'] = get_use_case_data_batch_conversations(
        batch_folder, max_workers=max_workers, limit_items_to_first_n=LIMIT_ITEMS_TO_FIRST_N
    )
    # Save progress after each item
    with open(progress_backup_path, 'wb') as backup_file:
        pickle.dump(batch_results, backup_file)
    
    print("  Generating areas of focus data...")
    batch_results[i]['areas_of_focus_data'] = get_areas_of_focus_data_batch_conversations(
        batch_folder, max_workers=max_workers, limit_items_to_first_n=LIMIT_ITEMS_TO_FIRST_N
    )
    # Save progress after each item
    with open(progress_backup_path, 'wb') as backup_file:
        pickle.dump(batch_results, backup_file)

print('DONE.')

Processing batch 1 of 4
  Generating behavioural tags...


Processing Conversations: 100%|██████████| 50/50 [00:30<00:00,  1.62it/s]


  Generating use case data...


100%|██████████| 50/50 [00:28<00:00,  1.75it/s]


  Generating areas of focus data...


...

  Generating use case data...


100%|██████████| 50/50 [00:26<00:00,  1.91it/s]


  Generating areas of focus data...


Processing Conversations: 100%|██████████| 50/50 [00:10<00:00,  4.68it/s]

DONE.


In [29]:
import pandas as pd

# Function to flatten the batch_results into a list of dictionaries
def flatten_batch_results(batch_results):
    flattened_data = []
    for batch_id, analytics_types in batch_results.items():
        for analytics_type, analytics_items in analytics_types.items():
            for item in analytics_items:
                item_data = {
                    'batch_id': batch_id,
                    'colab_link': item['colab_link'],
                    analytics_type: {k: v for k, v in item.items() if k not in ['colab_link', 'id']}
                }
                flattened_data.append(item_data)
    return flattened_data

# Flatten the batch_results
flattened_data = flatten_batch_results(batch_results)

# Create a dataframe from the flattened data
batch_results_df = pd.DataFrame(flattened_data)

# Function to extend the delivered_df with analytics data
def extend_delivered_df(delivered_df, batch_results_df):
    for analytics_type in item_types:
        analytics_df = batch_results_df[['batch_id', 'colab_link', analytics_type]].dropna()
        # Merge on 'colab_link' and exclude 'batch_id' and 'colab_link' from the right dataframe
        merged_df = delivered_df.merge(analytics_df, left_on='task_link', right_on='colab_link', how='left', suffixes=('', '_right'))
        # Drop the '_right' columns after merging
        for col in ['batch_id_right', 'colab_link_right']:
            if col in merged_df.columns:
                merged_df.drop(columns=[col], inplace=True)
        delivered_df = merged_df
    return delivered_df

# Extend the delivered_df with the batch_results_df
extended_delivered_df = extend_delivered_df(delivered_df, batch_results_df)

# Filter the extended dataframe to show only rows with non-null analytics data
filtered_extended_delivered_df = extended_delivered_df[
    extended_delivered_df.filter(like='_data').notnull().any(axis=1)
]

filtered_extended_delivered_df


,task_link,jsonl_link,metadata__topic,number_of_turns,duration_mins,batch_id,colab_link,behavioural_tags,use_case_data,areas_of_focus_data
1,https://colab.research.google.com/drive/1G_d7Z...,https://drive.google.com/file/d/1ql2hECkLQG3pI...,algorithms > by_data_structure > strings,3,60,1,https://colab.research.google.com/drive/1G_d7Z...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The user is utilizing the Assista...,"{'top_level': 'General coding help', 'sub_leve..."
2,https://colab.research.google.com/drive/1vYEPg...,https://drive.google.com/file/d/1YMkZcyiEqpqo3...,algorithms > by_data_structure > strings,3,50,1,https://colab.research.google.com/drive/1vYEPg...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The user utilizes the Assistant t...,"{'top_level': 'Basic scripting', 'sub_level': ..."
14,https://colab.research.google.com/drive/1Awxj_...,https://drive.google.com/file/d/1YyfStILNQT6ei...,algorithms > by_data_structure > trees,1,30,1,https://colab.research.google.com/drive/1Awxj_...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The user utilizes the Assistant t...,"{'top_level': 'Basic scripting', 'sub_level': ..."
33,https://colab.research.google.com/drive/1Iz1hQ...,https://drive.google.com/file/d/1dp99pw8zVR327...,algorithms > by_topic > bit_manipulation,1,10,1,https://colab.research.google.com/drive/1Iz1hQ...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The user is utilizing the Assista...,"{'top_level': 'Basic scripting', 'sub_level': ..."
35,https://colab.research.google.com/drive/1jV9aG...,https://drive.google.com/file/d/1KJl6u4-aEcspl...,algorithms > by_topic > divide_and_conquer,1,10,1,https://colab.research.google.com/drive/1jV9aG...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The user is utilizing the Assista...,"{'top_level': 'Basic scripting', 'sub_level': ..."
...,...,...,...,...,...,...,...,...,...,...
1898,https://colab.research.google.com/drive/1v-ZVO...,https://drive.google.com/file/d/1yxKwC7awFqqLb...,python_language_and_scripting > modules_and_pa...,2,10,4,https://colab.research.google.com/drive/1v-ZVO...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The user is utilizing the Assista...,"{'top_level': 'Basic scripting', 'sub_level': ..."
1903,https://colab.research.google.com/drive/1VHv-H...,https://drive.google.com/file/d/1R7lfSNvmxI9Mp...,algorithms > by_data_structure > linked_lists,2,20,4,https://colab.research.google.com/drive/1VHv-H...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The user is utilizing the Assista...,"{'top_level': 'Basic scripting', 'sub_level': ..."
1904,https://colab.research.google.com/drive/1VKZ5D...,https://drive.google.com/file/d/1HJrgO9ctwibvo...,python_language_and_scripting > object_oriente...,4,65,4,https://colab.research.google.com/drive/1VKZ5D...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The user is utilizing the Assista...,"{'top_level': 'Basic scripting', 'sub_level': ..."
1920,https://colab.research.google.com/drive/1x5G5Z...,https://drive.google.com/file/d/1thL8SsovjnkuV...,python_language_and_scripting > memory_managem...,5,55,4,https://colab.research.google.com/drive/1x5G5Z...,{'behaviours': [{'top_level': 'Continuation Fo...,{'summary': 'The User is utilizing the Assista...,"{'top_level': 'General coding help', 'sub_leve..."


In [30]:

def extract_summary(data):
    return data.get('summary', '') if pd.notnull(data) else ''

def extract_tags(data):
    return '||'.join(data.get('tags', [])) if pd.notnull(data) else ''

expanded_enriched_df = filtered_extended_delivered_df.copy()
expanded_enriched_df['use_case_summary'] = expanded_enriched_df['use_case_data'].map(extract_summary)
expanded_enriched_df['use_case_tags'] = expanded_enriched_df['use_case_data'].map(extract_tags)
expanded_enriched_df.drop(columns=['use_case_data'], inplace=True)

def extract_area_of_focus(data):
    if pd.notnull(data):
        return {
            'area_of_focus_top_level': data.get('top_level', ''),
            'area_of_focus_sub_level': data.get('sub_level', ''),
            'area_of_focus_detailed_level': data.get('detailed_level', ''),
            'area_of_focus_custom_category': data.get('new_filled_category', False)
        }
    else:
        return {
            'area_of_focus_top_level': '',
            'area_of_focus_sub_level': '',
            'area_of_focus_detailed_level': '',
            'area_of_focus_custom_category': False
        }

areas_of_focus_columns = expanded_enriched_df['areas_of_focus_data'].map(extract_area_of_focus).apply(pd.Series)
expanded_enriched_df = pd.concat([expanded_enriched_df, areas_of_focus_columns], axis=1)
expanded_enriched_df.drop(columns=['areas_of_focus_data'], inplace=True)

def extract_behavioural_tags(data):
    if pd.notnull(data) and 'behaviours' in data:
        tags = []
        for behaviour in data['behaviours']:
            tags.append(
                f"{behaviour.get('top_level', '')}->{behaviour.get('sub_level', '')};;"
                f"{behaviour.get('detailed_level', '')};;"
                f"{behaviour.get('new_filled_category', False)}"
            )
        return '||'.join(tags)
    else:
        return ''

behavioural_tags_column = expanded_enriched_df['behavioural_tags'].map(extract_behavioural_tags)
expanded_enriched_df['user_behaviour_tags'] = behavioural_tags_column
expanded_enriched_df.drop(columns=['behavioural_tags'], inplace=True)
cols = list(expanded_enriched_df.columns)
cols.append(cols.pop(cols.index('area_of_focus_custom_category')))
expanded_enriched_df = expanded_enriched_df[cols]

filename = f'batches{".".join(map(str, BATCH_IDS))}_'
filename += f'{"full" if LIMIT_ITEMS_TO_FIRST_N is None else str(LIMIT_ITEMS_TO_FIRST_N) + "items"}'
filename += '-use_case-area_of_focus-behaviour_tagging.csv'
expanded_enriched_df.to_csv(DATA_DIR + filename)
expanded_enriched_df

,task_link,jsonl_link,metadata__topic,number_of_turns,duration_mins,batch_id,colab_link,use_case_summary,use_case_tags,area_of_focus_top_level,area_of_focus_sub_level,area_of_focus_detailed_level,user_behaviour_tags,area_of_focus_custom_category
1,https://colab.research.google.com/drive/1G_d7Z...,https://drive.google.com/file/d/1ql2hECkLQG3pI...,algorithms > by_data_structure > strings,3,60,1,https://colab.research.google.com/drive/1G_d7Z...,The user is utilizing the Assistant to improve...,software development||coding assistance||probl...,General coding help,Explain complex code snippets,Modifying a function to handle special cases,Continuation Follow up->Incrementally Build;;U...,False
2,https://colab.research.google.com/drive/1vYEPg...,https://drive.google.com/file/d/1YMkZcyiEqpqo3...,algorithms > by_data_structure > strings,3,50,1,https://colab.research.google.com/drive/1vYEPg...,The user utilizes the Assistant to obtain codi...,coding assistance||Python||string manipulation...,Basic scripting,Write simple code in Python,Provide code snippets with conditions and expl...,Continuation Follow up->Incrementally Build;;U...,False
14,https://colab.research.google.com/drive/1Awxj_...,https://drive.google.com/file/d/1YyfStILNQT6ei...,algorithms > by_data_structure > trees,1,30,1,https://colab.research.google.com/drive/1Awxj_...,The user utilizes the Assistant to obtain spec...,programming assistance||code example||algorith...,Basic scripting,Write simple code in Python,In-order traversal on a binary tree,Continuation Follow up->Incrementally Build;;A...,False
33,https://colab.research.google.com/drive/1Iz1hQ...,https://drive.google.com/file/d/1dp99pw8zVR327...,algorithms > by_topic > bit_manipulation,1,10,1,https://colab.research.google.com/drive/1Iz1hQ...,The user is utilizing the Assistant to underst...,programming assistance||bit manipulation||Pyth...,Basic scripting,Write simple code in Python,Bit manipulation operations,Continuation Follow up->Incrementally Build;;A...,False
35,https://colab.research.google.com/drive/1jV9aG...,https://drive.google.com/file/d/1KJl6u4-aEcspl...,algorithms > by_topic > divide_and_conquer,1,10,1,https://colab.research.google.com/drive/1jV9aG...,The user is utilizing the Assistant to learn a...,education||programming||algorithm||Python||bin...,Basic scripting,Write simple code in Python,Implementing binary search algorithm,Continuation Follow up->Supplement/Extend;;Ass...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898,https://colab.research.google.com/drive/1v-ZVO...,https://drive.google.com/file/d/1yxKwC7awFqqLb...,python_language_and_scripting > modules_and_pa...,2,10,4,https://colab.research.google.com/drive/1v-ZVO...,The user is utilizing the Assistant to learn a...,Python programming||Code optimization||Import ...,Basic scripting,Write simple code in Python,Importing specific functions from modules,Continuation Follow up->Incrementally Build;;A...,False
1903,https://colab.research.google.com/drive/1VHv-H...,https://drive.google.com/file/d/1R7lfSNvmxI9Mp...,algorithms > by_data_structure > linked_lists,2,20,4,https://colab.research.google.com/drive/1VHv-H...,The user is utilizing the Assistant to develop...,Python Programming||Data Structures||Linked Li...,Basic scripting,Write simple code in Python,Implementing and optimizing data structures,Continuation Follow up->Incrementally Build;;A...,False
1904,https://colab.research.google.com/drive/1VKZ5D...,https://drive.google.com/file/d/1HJrgO9ctwibvo...,python_language_and_scripting > object_oriente...,4,65,4,https://colab.research.google.com/drive/1VKZ5D...,The user is utilizing the Assistant to underst...,Python Encapsulation||Getters and Setters||Inh...,Basic scripting,Write simple code in Python,Encapsulation with getters and setters,Continuation Follow up->Incrementally Build;;U...,False
1920,https://colab.research.google.com/drive/1x5G5Z...,https://drive.google.com/file/d/1thL8SsovjnkuV...,python_language_and_scripti

In [50]:
expanded_enriched_df = filtered_extended_delivered_df.copy()
expanded_enriched_df['use_case_summary'] = expanded_enriched_df['use_case_data'].map(extract_summary)
def expand_tags(row):
    tags = row['use_case_data'].get('tags', [])
    task_link = row['task_link']
    return [{'task_link': task_link, 'tag': tag} for tag in tags]

tags_data = expanded_enriched_df.apply(expand_tags, axis=1).explode()
tags_data = pd.DataFrame(tags_data.tolist())
tags_data

,task_link,tag
0,https://colab.research.google.com/drive/1G_d7Z...,software development
1,https://colab.research.google.com/drive/1G_d7Z...,coding assistance
2,https://colab.research.google.com/drive/1G_d7Z...,problem-solving
3,https://colab.research.google.com/drive/1G_d7Z...,code refinement
4,https://colab.research.google.com/drive/1G_d7Z...,collaborative programming
...,...,...
1009,https://colab.research.google.com/drive/14TFOI...,writing assistance
1010,https://colab.research.google.com/drive/14TFOI...,article preparation
1011,https://colab.research.google.com/drive/14TFOI...,Python error handling
1012,https://colab.research.google.com/drive/14TFOI...,information gathering


In [57]:

def extract_summary(data):
    return data.get('summary', '') if pd.notnull(data) else ''


expanded_enriched_df = filtered_extended_delivered_df.copy()
expanded_enriched_df['use_case_summary'] = expanded_enriched_df['use_case_data'].map(extract_summary)

def expand_tags(row):
    tags = row['use_case_data'].get('tags', [])
    task_link = row['task_link']
    return [{'task_link': task_link, 'tag': tag} for tag in tags]

tags_data = expanded_enriched_df.apply(expand_tags, axis=1).explode()
tags_data = pd.DataFrame(tags_data.tolist())

expanded_enriched_df.drop(columns=['use_case_data'], inplace=True)

def extract_area_of_focus(data):
    if pd.notnull(data):
        return {
            'area_of_focus_top_level': data.get('top_level', ''),
            'area_of_focus_sub_level': data.get('sub_level', ''),
            'area_of_focus_detailed_level': data.get('detailed_level', ''),
            'area_of_focus_custom_category': data.get('new_filled_category', False)
        }
    else:
        return {
            'area_of_focus_top_level': '',
            'area_of_focus_sub_level': '',
            'area_of_focus_detailed_level': '',
            'area_of_focus_custom_category': False
        }

areas_of_focus_columns = expanded_enriched_df['areas_of_focus_data'].map(extract_area_of_focus).apply(pd.Series)
expanded_enriched_df = pd.concat([expanded_enriched_df, areas_of_focus_columns], axis=1)
expanded_enriched_df.drop(columns=['areas_of_focus_data'], inplace=True)

def extract_behavioural_tags(row):
    data = row['behavioural_tags']
    if pd.notnull(data) and 'behaviours' in data:
        tags = data['behaviours']
        return [
            {
                'task_link': row['task_link'],
                'top_level': tag.get('top_level', ''),
                'sub_level': tag.get('sub_level', ''),
                'detailed_level': tag.get('detailed_level', ''),
                'custom_category': tag.get('new_filled_category', False)
            }
            for tag in tags
        ]
    return []

behavioural_tags_data = expanded_enriched_df.apply(extract_behavioural_tags, axis=1).explode()
behavioural_tags_data = pd.DataFrame(behavioural_tags_data.tolist())


expanded_enriched_df.drop(columns=['behavioural_tags'], inplace=True)
cols = list(expanded_enriched_df.columns)
cols.append(cols.pop(cols.index('area_of_focus_custom_category')))
expanded_enriched_df = expanded_enriched_df[cols]

import os

def save_to_csv(df, dir_name, suffix):
    filename = f'{base_filename}-{suffix}.csv'
    path = os.path.join(dir_name, filename)
    df.to_csv(path)

base_filename = f'batches{".".join(map(str, BATCH_IDS))}_'
base_filename += f'{"full" if LIMIT_ITEMS_TO_FIRST_N is None else str(LIMIT_ITEMS_TO_FIRST_N) + "items"}'
from datetime import datetime
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
bundle_dir = f'{DATA_DIR}/{base_filename}_{timestamp}'
os.makedirs(bundle_dir, exist_ok=True)

save_to_csv(expanded_enriched_df, bundle_dir, 'expanded_enriched_df')
save_to_csv(tags_data, bundle_dir, 'tags_data')
save_to_csv(behavioural_tags_data, bundle_dir, 'behavioural_tags_data')


,task_link,tag
0,https://colab.research.google.com/drive/1G_d7Z...,software development
1,https://colab.research.google.com/drive/1G_d7Z...,coding assistance
2,https://colab.research.google.com/drive/1G_d7Z...,problem-solving
3,https://colab.research.google.com/drive/1G_d7Z...,code refinement
4,https://colab.research.google.com/drive/1G_d7Z...,collaborative programming
...,...,...
1009,https://colab.research.google.com/drive/14TFOI...,writing assistance
1010,https://colab.research.google.com/drive/14TFOI...,article preparation
1011,https://colab.research.google.com/drive/14TFOI...,Python error handling
1012,https://colab.research.google.com/drive/14TFOI...,information gathering


,task_link,top_level,sub_level,detailed_level,new_filled_category
0,https://colab.research.google.com/drive/1G_d7Z...,Continuation Follow up,Incrementally Build,User incrementally builds upon the assistant's...,False
1,https://colab.research.google.com/drive/1G_d7Z...,Continuation Follow up,Request for clarification/elaboration,Assistant requests clarification on how to mod...,False
2,https://colab.research.google.com/drive/1G_d7Z...,Respond to Assistant,Answer a question,User provides a detailed answer to the assista...,False
3,https://colab.research.google.com/drive/1G_d7Z...,Continuation Follow up,Supplement/Extend,Assistant asks for further extension of the fu...,False
4,https://colab.research.google.com/drive/1vYEPg...,Continuation Follow up,Incrementally Build,User asks to incrementally build on the provid...,False
...,...,...,...,...,...
686,https://colab.research.google.com/drive/14TFOI...,Continuation Follow up,Request for clarification/elaboration,User requests for clarification on handling mu...,False
687,https://colab.research.google.com/drive/14TFOI...,Respond to Assistant,Answer a question,Assistant answers user's questions and provide...,False
688,https://colab.research.google.com/drive/14TFOI...,User actions,User makes mistakes,User does not make any mistakes in the convers...,False
689,https://colab.research.google.com/drive/14TFOI...,User emotional state,User is confused,User is not confused but seeks guidance on spe...,False


In [31]:
print(filtered_extended_delivered_df['use_case_data'].iloc[0])
print(filtered_extended_delivered_df['areas_of_focus_data'].iloc[0])
print(filtered_extended_delivered_df['behavioural_tags'].iloc[0])


{'summary': "The user is utilizing the Assistant to improve a programming function that reverses strings while maintaining the position of special characters. The user seeks guidance on modifying the code to achieve this functionality and engages in a collaborative coding session with the Assistant. The conversation involves sharing code snippets, discussing potential solutions, and refining the function to meet the specified requirements. The user's use case demonstrates a practical application of the Assistant's capabilities in software development and problem-solving.", 'tags': ['software development', 'coding assistance', 'problem-solving', 'code refinement', 'collaborative programming']}
{'top_level': 'General coding help', 'sub_level': 'Explain complex code snippets', 'detailed_level': 'Modifying a function to handle special cases', 'new_filled_category': False}
{'behaviours': [{'top_level': 'Continuation Follow up', 'sub_level': 'Incrementally Build', 'detailed_level': "User inc

In [19]:
delivered4 = download_parse_delivered_into_jsonl([1,2,3,4], no_work=True)
delivered_df = delivered4['delivered_df']
conversations = delivered4['conversations']

In [20]:
data = [
    # User Behaviors
    ("User", "Emotional Response", "Confusion"),
    ("User", "Emotional Response", "Frustration"),
    ("User", "Behavior", "Self-Contradiction"),
    ("User", "Behavior", "Error Identification"),
    ("User", "Behavior", "Making Errors"),
    ("User", "Feedback", "Satisfaction Expression"),
    ("User", "Feedback", "Compliment Offering"),
    ("User", "Intent", "Information Seeking"),
    ("User", "Intent", "Transactional"),
    ("User", "Intent", "Casual Interaction"),
    ("User", "Conversation Progression", "Building Upon Discussion"),
    ("User", "Conversation Progression", "Extending Scope"),
    ("User", "Conversation Progression", "Seeking Clarification"),
    ("User", "Conversation Progression", "Exploring Alternatives"),
    ("User", "Conversation Direction", "Topic Shift"),
    ("User", "Conversation Direction", "Goal Reassessment"),
    
    # Assistant Behaviors
    ("Assistant", "Interaction", "Providing Information"),
    ("Assistant", "Interaction", "Concluding Interaction"),
    ("Assistant", "Interaction", "Resolving Ambiguities"),
    ("Assistant", "Performance", "Misunderstanding"),
    ("Assistant", "Performance", "Accurate Information"),
    ("Assistant", "Performance", "Issue Escalation"),
    ("Assistant", "Technical Support", "Issue Identification"),
    ("Assistant", "Technical Support", "Support Provision"),
    
    # Conversation Closure
    ("General", "Conversation Closure", "Satisfied Conclusion"),
    ("General", "Conversation Closure", "Unsatisfied Conclusion"),
    ("General", "Conversation Closure", "Inactive Termination"),
    
    # Other categories can be added under "General" or other relevant top-level categories
]


import pandas as pd

behavioural_tags_df2 = pd.DataFrame(data, columns=["Top_Level_Category", "Sub_Level_Category", "Detailed_Level_Category"])
behavioural_tags_df2






,Top_Level_Category,Sub_Level_Category,Detailed_Level_Category
0,User,Emotional Response,Confusion
1,User,Emotional Response,Frustration
2,User,Behavior,Self-Contradiction
3,User,Behavior,Error Identification
4,User,Behavior,Making Errors
5,User,Feedback,Satisfaction Expression
6,User,Feedback,Compliment Offering
7,User,Intent,Information Seeking
8,User,Intent,Transactional
9,User,Intent,Casual Interaction
